In [ ]:
!pip install fbprophet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!sudo apt install tesseract-ocr

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


In [ ]:
!pip install pytesseract --quiet

In [ ]:
!pip install streamlit --quiet

In [ ]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pandas_datareader import data as pdr

import yfinance as yf
yf.pdr_override() # <== that's all it takes :-)

# download dataframe
data = pdr.get_data_yahoo("SPY", start="2017-01-01", end="2017-04-30")
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-01-03,225.039993,225.830002,223.880005,225.240005,205.509125,91366500
2017-01-04,225.619995,226.750000,225.610001,226.580002,206.731705,78744400
2017-01-05,226.270004,226.580002,225.479996,226.399994,206.567505,78379000
2017-01-06,226.529999,227.750000,225.899994,227.210007,207.306488,71559900
2017-01-09,226.910004,227.070007,226.419998,226.460007,206.622208,46939700
...,...,...,...,...,...,...
2017-04-24,237.179993,237.410004,234.559998,237.169998,217.335419,119209900
2017-04-25,237.910004,238.949997,237.809998,238.550003,218.600021,76698300
2017-04-26,238.509995,239.529999,238.350006,238.399994,218.462601,84702500


In [ ]:
%%writefile app4.py
import streamlit as st
from datetime import date

import yfinance as yf
from fbprophet import Prophet
from fbprophet.plot import plot_plotly
from plotly import graph_objs as go

START = "2010-01-01"
TODAY = date.today().strftime("%Y-%m-%d")

st.title('Stock Market Prediction')
st.image('/content/stock.png',width=800)

st.write("Alphabet Inc.(Google) --      GOOG")
st.write("Apple Inc --                  AAPL")
st.write("Microsoft Corporation --      MSFT")
st.write("Amazon.com, Inc. --           AMZN")
st.write("Meta Platforms, Inc. --       FB")
st.write("Tesla, Inc. --                TSLA")

stocks = ('GOOG', 'AAPL', 'MSFT', 'AMZN','FB','TSLA')
selected_stock = st.selectbox('Select dataset for prediction', stocks)

n_years = st.slider('Years of prediction:', 1, 4)
period = n_years * 365


@st.cache
def load_data(ticker):
    data = yf.download(ticker, START, TODAY)
    data.reset_index(inplace=True)
    return data

	
data_load_state = st.text('Loading data...')
data = load_data(selected_stock)
data_load_state.text('Loading data... done!')

st.subheader('Raw data')
st.write(data.tail())

# Plot raw data
def plot_raw_data():
	fig = go.Figure()
	fig.add_trace(go.Scatter(x=data['Date'], y=data['Open'], name="stock_open"))
	fig.add_trace(go.Scatter(x=data['Date'], y=data['Close'], name="stock_close"))
	fig.layout.update(title_text='Time Series data with Rangeslider', xaxis_rangeslider_visible=True)
	st.plotly_chart(fig)
	
plot_raw_data()

# Predict forecast with Prophet.
df_train = data[['Date','Close']]
df_train = df_train.rename(columns={"Date": "ds", "Close": "y"})

m = Prophet()
m.fit(df_train)
future = m.make_future_dataframe(periods=period)
forecast = m.predict(future)

# Show and plot forecast
st.subheader('Forecast data')
st.write(forecast.tail())
    
st.write(f'Forecast plot for {n_years} years')
fig1 = plot_plotly(m, forecast)
st.plotly_chart(fig1)

st.write("Forecast components")
fig2 = m.plot_components(forecast)
st.write(fig2)

Writing app4.py


In [ ]:
!streamlit run app4.py & npx localtunnel --port 8501

2022-05-25 06:14:20.883 INFO    numexpr.utils: NumExpr defaulting to 2 threads.
npx: installed 22 in 5.717s

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://104.199.253.248:8501

your url is: https://petite-poems-stay-104-199-253-248.loca.lt
[*********************100%***********************]  1 of 1 completed
2022-05-25 06:15:47.691 Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
Initial log joint probability = -35.2917
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       8469.02    0.00402524       2142.29      0.3363      0.3363      118   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       8551.89     0.0434215       745.961           1           1      230   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299        85